# Practical case - Food Orders

### Statement (written in spanish) 

Para este ejercicio se va a imaginar que se trabaja para una empresa de envíos de comida, presente
en todo el territorio nacional, con miles de pedidos cada día. Dicha empresa tiene un fichero histórico
con todas las peticiones de comida que los clientes han realizado mediante el chat de su web en los
últimos meses. Necesitan analizar en tiempo real qué comidas están pidiendo los usuarios y qué
ingredientes tenían, ya que en la cadena de stock de alimentos es necesario realizar una previsión para
no quedarse sin platos cocinados. 

Se ha calculado que el impacto en las ventas cada vez que uno de
los platos deja de estar disponible es del 7% de pérdidas en esa semana, debido al abandono de la web
de pedidos por parte del cliente. Por tanto, es de vital importancia poder realizar automáticamente
estimaciones al respecto.

El objetivo es programar una función que reciba como input un texto de usuario y devuelva los
fragmentos de texto (chunks) que hagan referencia a las comidas y cantidades que ha solicitado. No es
necesario, ni es el objetivo de este ejercicio, construir un clasificador de intención previo a esta
función, sino simplemente una función que presuponemos recibe una frase con la intención
`Pedir_comida`. Tampoco es objetivo normalizar la salida (por ej.: no es necesario convertir 'tres' a '3'
ni 'pizzas' a 'pizza'). Es, por tanto, un ejercicio de mínimos.

    Por ejemplo: “quiero 3 bocadillos de anchoas y 2 pizzas” →
    [
        {comida:'bocadillo', ingrediente:'anchoas', cantidad:3},
        {comida:'pizza', ingrediente:'null', cantidad:2}
    ]
    
Por tanto, la salida de la función será un array con diccionarios de 2 elementos (`comida` y `cantidad`).
Cuando una cantidad no sea detectada, se pondrá su valor a '1' como valor por defecto.

Este ejercicio hay que hacerlo con textos de entrenamiento en español, pero teniendo en cuenta que
la precisión de los POS taggers en castellano de NLTK es muy mala. Por tanto, el alumno no debe
frustrarse por no obtener buenos resultados, como hemos dicho anteriormente se trata simplemente de
un ejercicio teórico y podemos suponer que, con un mejor analizador, podríamos obtener mejores
resultados.

Para llevar a cabo la práctica, deberá construirse una cadena NLP con NLTK, con los siguientes
elementos:
    - segmentación de frases,
    - tokenización,
    - POS tagger (analizador mofológico para el español).

A continuación, los POS tags obtenidos serán usados por el `RegexParser`, el `UnigramParser`, el
`BigramParser` y el `NaiveBayesClassifier`.

In [1]:
%pylab
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

import numpy as np

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


### Import NLTK and Spanish CESS Corpus 

In [1]:
import nltk
from nltk.corpus import cess_esp

# Load all tagged sentences of Spanish CESS corpus
sents = cess_esp.tagged_sents()

## POS tagger training 

Before making the several versions required by that practical case, we have to train a POS tagger for Spanish CESS corpus. For that, we are going to use the `HiddenMarkovModelTagger` tagger, because that tagger is more complete than `UnigramTagger`, `BigramTagger` and `TrigramTagger` taggers.

At first, we will make the train (90%) and test (10%) datasets and evaluate if the trained model is not overfitting:

In [7]:
training = []
testing = []

for i in range(len(sents)) :
    if i % 10 :
        training.append(sents[i])
    else :
        testing.append(sents[i])

In [12]:
print('len(sents) =', len(sents))
print('len(training) =', len(training))
print('len(testing) =', len(testing))

len(sents) = 6030
len(training) = 5427
len(testing) = 603


In [8]:
# Import HiddenMarkovModelTagger
from nltk.tag.hmm import HiddenMarkovModelTagger

# Create the Spanish POS tagger using HMM Tagger
spanish_pos_tagger = HiddenMarkovModelTagger.train(training)

# Evaluate that tagger
print('Evaluation:', spanish_pos_tagger.evaluate(testing)*100)

Evaluation: 89.88905831011094


As we can see, that model is not overfitting and its evaluation is good. So, we will use that POS tagger for our tagging.

## Get Part-of-Speech Tagset of Spanish CESS Corpus

Before using `RegexParser`, we have to know how are the Spanish tags provided by our tagger. 

In [ ]:
# Create a set which will contain all tags
tagset = set()

for sent in sents :
    tagset.update([ tag for (word, tag) in sent ])

In [50]:
# Print sorted tagset
print(sorted(tagset))

['Faa', 'Fat', 'Fc', 'Fd', 'Fe', 'Fg', 'Fh', 'Fia', 'Fit', 'Fp', 'Fpa', 'Fpt', 'Fs', 'Fx', 'Fz', 'I', 'W', 'X', 'Y', 'Z', 'Zm', 'Zp', 'ao0fp0', 'ao0fs0', 'ao0mp0', 'ao0ms0', 'aq00000', 'aq0cn0', 'aq0cp0', 'aq0cs0', 'aq0fp0', 'aq0fpp', 'aq0fs0', 'aq0fsp', 'aq0mp0', 'aq0mpp', 'aq0ms0', 'aq0msp', 'cc', 'cs', 'da0fp0', 'da0fs0', 'da0mp0', 'da0ms0', 'da0ns0', 'dd0cp0', 'dd0cs0', 'dd0fp0', 'dd0fs0', 'dd0mp0', 'dd0ms0', 'de0cn0', 'di0cp0', 'di0cs0', 'di0fp0', 'di0fs0', 'di0mp0', 'di0ms0', 'dn0cp0', 'dn0cs0', 'dn0fp0', 'dn0fs0', 'dn0mp0', 'dn0ms0', 'dp1cps', 'dp1css', 'dp1fpp', 'dp1fsp', 'dp1mpp', 'dp1msp', 'dp1mss', 'dp2cps', 'dp2css', 'dp3cp0', 'dp3cs0', 'dp3fs0', 'dp3mp0', 'dp3ms0', 'dt0cn0', 'dt0fs0', 'dt0ms0', 'i', 'nc00000', 'nccn000', 'nccp000', 'nccs000', 'ncfn000', 'ncfp000', 'ncfs000', 'ncmn000', 'ncmp000', 'ncms000', 'np00000', 'np0000a', 'np0000l', 'np0000o', 'np0000p', 'p0000000', 'p010p000', 'p010s000', 'p020s000', 'p0300000', 'pd0cp000', 'pd0cs000', 'pd0fp000', 'pd0fs000', 'pd0m

With that, we cannot know what is each tag. However, we can see that some tags are similar. For that, we will show some examples for each tag begining:

In [69]:
[ (word, tag) for (word, tag) in sents[100] if tag.startswith('a') ]

[('dedicada', 'aq0fsp'),
 ('democrático', 'aq0ms0'),
 ('político', 'aq0ms0'),
 ('fundamentales', 'aq0cp0'),
 ('eficiente', 'aq0cs0'),
 ('participativa', 'aq0fs0')]

In [71]:
[ (word, tag) for (word, tag) in sents[100] if tag.startswith('d') ]

[('la', 'da0fs0'),
 ('la', 'da0fs0'),
 ('el', 'da0ms0'),
 ('el', 'da0ms0'),
 ('el', 'da0ms0'),
 ('el', 'da0ms0'),
 ('los', 'da0mp0'),
 ('las', 'da0fp0'),
 ('la', 'da0fs0'),
 ('una', 'di0fs0')]

In [72]:
[ (word, tag) for (word, tag) in sents[100] if tag.startswith('n') ]

[('La_Declaración_de_Viña_del_Mar', 'np0000a'),
 ('Gobernabilidad', 'ncfs000'),
 ('democracia', 'ncfs000'),
 ('compromiso', 'ncms000'),
 ('sistema', 'ncms000'),
 ('Estado_de_Derecho', 'np0000a'),
 ('pluralismo', 'ncms000'),
 ('derechos_humanos', 'ncmp000'),
 ('libertades', 'ncfp000'),
 ('marco', 'ncms000'),
 ('gobernabilidad', 'ncfs000'),
 ('democracia', 'ncfs000')]

In [75]:
[ (word, tag) for (word, tag) in sents[300] if tag.startswith('p') ]

[('que', 'pr0cn000'), ('se', 'p0300000'), ('que', 'pr0cn000')]

Now, we are going to use our POS tagger with a example for seeing how it tags that example:

In [120]:
sentence = 'me pones una pizza de cuatro quesos?, un buen bocata con pepinillos y chorizo'

tokens = nltk.word_tokenize(sentence)

tagged = spanish_pos_tagger.tag(tokens)
tagged

[('me', 'pp1cs000'),
 ('pones', 'vmip3s0'),
 ('una', 'di0fs0'),
 ('pizza', 'ncfs000'),
 ('de', 'sps00'),
 ('cuatro', 'dn0cp0'),
 ('quesos', 'ncmp000'),
 ('?', 'Fit'),
 (',', 'Fc'),
 ('un', 'di0ms0'),
 ('buen', 'aq0ms0'),
 ('bocata', 'ncms000'),
 ('con', 'sps00'),
 ('pepinillos', 'np0000l'),
 ('y', 'cc'),
 ('chorizo', 'sn.e-SUJ')]

With that, we can know what is the meaning of some tags:
    
    
| Tag Begining | Meaning |
|--------------|---------|
|       a      | adjetive|
| d | determiner |
| n | noun |

That is useful for create our `RegexParser`:

## Version 1: `RegexParser`

In [344]:
sentence = 'yo quiero 3 bocadillos con pimiento, 3 pizzas y ensalada'

tokens = nltk.word_tokenize(sentence)

tagged = spanish_pos_tagger.tag(tokens)
tagged


# Define the grammar
grammar = r"""
    Comida: {<s?n.*>}      
    Cantidad: {<[Zd].*>}   
"""

#Ingrediente: {<s.*>+<Comida>}
#Pedido: {<Cantidad>?<Comida><Ingrediente>?}

# Create the RegexParser
regex_parser = nltk.RegexpParser(grammar)

# Chunk the tagged sentence
chunked = regex_parser.parse(tagged)
print(chunked)
nltk.chunk.tree2conlltags(chunked)

(S
  yo/pp1csn00
  quiero/vmip1s0
  (Cantidad 3/di0fp0)
  (Comida bocadillos/ncfp000)
  con/sps00
  (Comida pimiento/np0000l)
  ,/Fc
  (Cantidad 3/Z)
  (Comida pizzas/ncmp000)
  y/cc
  (Comida ensalada/sn.e-SUJ))


[('yo', 'pp1csn00', 'O'),
 ('quiero', 'vmip1s0', 'O'),
 ('3', 'di0fp0', 'B-Cantidad'),
 ('bocadillos', 'ncfp000', 'B-Comida'),
 ('con', 'sps00', 'O'),
 ('pimiento', 'np0000l', 'B-Comida'),
 (',', 'Fc', 'O'),
 ('3', 'Z', 'B-Cantidad'),
 ('pizzas', 'ncmp000', 'B-Comida'),
 ('y', 'cc', 'O'),
 ('ensalada', 'sn.e-SUJ', 'B-Comida')]

In [340]:
delivery = []
dic = {}
default_cantidad = 1

for chunk in [ chunk for chunk in nltk.chunk.tree2conlltags(chunked) if 'B' in chunk[2] ] :
    w, t, c = chunk
    
    if 'cantidad' not in dic :
        if c == 'B-Cantidad' :
            dic['cantidad'] = int(w)
        else :
            dic['cantidad'] = default_cantidad
    
    if 'comida' not in dic and c == 'B-Comida' :
        dic['comida'] = w
        delivery.append(dic)
        dic = {}
    
delivery

[{'cantidad': 3, 'comida': 'bocadillos'},
 {'cantidad': 1, 'comida': 'pimiento'},
 {'cantidad': 3, 'comida': 'pizzas'},
 {'cantidad': 1, 'comida': 'ensalada'}]